In [ ]:
# !pip install openai==0.28

In [ ]:
!pip install backoff

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import backoff #bilioteca para reexecutar o codigo caso ele caia em um erro determinado por nós, considerando que esse erro geralmente está relacioando a limite de taxa da API ou algo parecido
#ele espera um tempo para reexecutar
import os
from openai import OpenAI
import time
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sample_comments_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')

In [ ]:
client = OpenAI(api_key="")

##Criar **dataset**

In [ ]:
client.files.create(
  file=open("/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/dados90.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-BTDS46hzAvDeqVFdeLaDN9', bytes=113597, created_at=1735840730, filename='dados90.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-BTDS46hzAvDeqVFdeLaDN9",
  model="gpt-4o-2024-08-06"
)

FineTuningJob(id='ftjob-mWa5LOoeCuWCfVnYpWLDRyyo', created_at=1735840744, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-zKfgABWcJBjnw3FWo8tYit9c', result_files=[], seed=766854869, status='validating_files', trained_tokens=None, training_file='file-BTDS46hzAvDeqVFdeLaDN9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [ ]:
fine_tuning_job = client.fine_tuning.jobs.retrieve("ftjob-mWa5LOoeCuWCfVnYpWLDRyyo")
print(fine_tuning_job)

FineTuningJob(id='ftjob-mWa5LOoeCuWCfVnYpWLDRyyo', created_at=1735840744, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:personal::AlJxCMLn', finished_at=1735841480, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-zKfgABWcJBjnw3FWo8tYit9c', result_files=['file-L9U5AkZCgwDuoZx7wd1mZD'], seed=766854869, status='succeeded', trained_tokens=70620, training_file='file-BTDS46hzAvDeqVFdeLaDN9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}})


## Loading comments

In [ ]:
import re
import html
def pre_processamento (x):

  retorno = re.sub(r'http(s)?://\S+|www\.\S+', '', x)
  verdadeiro = re.search(r'&\S+', retorno)

  if verdadeiro:
        retorno = html.unescape(retorno)

  return retorno

In [ ]:
sample_comments_df.loc[:, 'Comentário'] = sample_comments_df.loc[:, 'Comentário'].apply(lambda x: pre_processamento(x))

In [ ]:
sample_comments_df.head()

,Unnamed: 0.1,Unnamed: 0,id,Comentário,class_result,total_agreement,gpt_prompt1,gpt_prompt2
0,0,0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1,2
1,1,1,hub93yq,Campeonato de quarto,1,1,1,1
2,2,2,hv1xsjp,Não entendi,1,1,1,1
3,3,3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0,0
4,4,4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1,1


In [ ]:
sample_comments_df.shape

(171, 8)

In [ ]:
comment_ids = sample_comments_df['id'].tolist()

In [ ]:
comments_list = sample_comments_df['Comentário'].tolist()

## GPT-4o ft

In [ ]:
DEFAULT_TEMPERATURE = 0 #quanto menor, maior será a precisão e menor a criatividade
DEFAULT_MAX_TOKENS = 800

In [ ]:
DELIMITER = "###"

In [ ]:

INSTRUCTIONS = f"""Você é um assistente que classifica comentários do Reddit em Português do Brasil (PT-BR) como "Positivo", "Negativo" ou "Neutro".
Você receberá o texto de um comentário e a sua tarefa é classificar o sentimento do texto fornecido. Use somente as informações abaixo para fazer a predição.

1. Para cada comentario se limite a escolher apenas uma dessas três opções, sem acrescentar texto explicativo e sem marcar outras opções que não sejam uma dessas três: "Positivo", "Negativo" ou "Neutro";
2. Marque somente como "Positivo" os comentários que tiver certeza, alta confiança de que tenham o sentimento positivo;
3. Marque somente como "Negativo" os comentários que tiver certeza, alta confiança de que tenham o sentimento negativo;
4. Marque somente como "Neutro" os comentários que tiver certeza, alta confiança de que tenham o sentimento neutro.
""".strip()


'''
INSTRUCTIONS = f"""You are an assistant that classifies Reddit comments in Brazilian Portuguese (PT-BR) as 'Positive', 'Negative', or 'Neutral'.
You will receive the text of a comment, and your task is to classify the sentiment of the provided text. Use only the information below to make the prediction.
For each comment, limit yourself to choosing only one of these three options, without adding explanatory text and without marking any options other than one of these three: 'Positive', 'Negative', or 'Neutral'.
    1. Mark only as 'Positive' the comments that you are certain, with high confidence, express a positive sentiment.
    2. Mark only as 'Negative' the comments that you are certain, with high confidence, express a negative sentiment.
    3. Mark only as 'Neutral' the comments that you are certain, with high confidence, express a neutral sentiment.
""".strip()
'''

'\nINSTRUCTIONS = f"""You are an assistant that classifies Reddit comments in Brazilian Portuguese (PT-BR) as \'Positive\', \'Negative\', or \'Neutral\'.\nYou will receive the text of a comment, and your task is to classify the sentiment of the provided text. Use only the information below to make the prediction.\nFor each comment, limit yourself to choosing only one of these three options, without adding explanatory text and without marking any options other than one of these three: \'Positive\', \'Negative\', or \'Neutral\'.\n    1. Mark only as \'Positive\' the comments that you are certain, with high confidence, express a positive sentiment.\n    2. Mark only as \'Negative\' the comments that you are certain, with high confidence, express a negative sentiment.\n    3. Mark only as \'Neutral\' the comments that you are certain, with high confidence, express a neutral sentiment.\n""".strip()\n'

In [ ]:
# @backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def send_prompt(prompt: str, model: str) -> str:

    messages = [
        {"role": "system", "content": INSTRUCTIONS},
        {"role": "user", "content": prompt}
    ]

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS
    )

    answer = resp.choices[0].message.content.strip()

    return answer

In [ ]:
len(comment_ids), len(comments_list)

(171, 171)

In [ ]:
predictions_sample = []

In [ ]:
processed_ids = {}

In [ ]:
len(processed_ids)

0

In [ ]:
%%time

c = 1
for c_id, comment in zip(comment_ids, comments_list):
    if c_id in set(processed_ids):
        continue
    prompt = f"Comentário: {comment}"

    answer = send_prompt(prompt=prompt, model='ft:gpt-4o-2024-08-06:personal::AlJ2QGgD')

    if c % 10 == 0 :
        print(f"collected predictions for {c} comments...")
        time.sleep(20)

    predictions_sample.append({
        'id': c_id,
        'text': comment,
        'prediction': answer
    })
    c += 1

collected predictions for 10 comments...
collected predictions for 20 comments...
collected predictions for 30 comments...
collected predictions for 40 comments...
collected predictions for 50 comments...
collected predictions for 60 comments...
collected predictions for 70 comments...
collected predictions for 80 comments...
collected predictions for 90 comments...
collected predictions for 100 comments...
collected predictions for 110 comments...
collected predictions for 120 comments...
collected predictions for 130 comments...
collected predictions for 140 comments...
collected predictions for 150 comments...
collected predictions for 160 comments...
collected predictions for 170 comments...
CPU times: user 3.93 s, sys: 332 ms, total: 4.26 s
Wall time: 7min 43s


In [ ]:
predictions_df = pd.DataFrame(predictions_sample)

In [ ]:
processed_ids = predictions_df['id'].unique().tolist()

In [ ]:
predictions_df.head()

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,Positivo
1,hub93yq,Campeonato de quarto,Neutro
2,hv1xsjp,Não entendi,Neutro
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",Negativo
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,Positivo


In [ ]:
predictions_df.shape

(171, 3)

In [ ]:
predictions_df['prediction'].value_counts(normalize=True)

,proportion
prediction,
Negativo,0.479532
Positivo,0.292398
Neutro,0.228070


In [ ]:
def rotulacao(text):
  if text == 'Negativo':
    return 0
  elif text == 'Positivo':
    return 2
  elif text == 'Neutro':
    return 1
  else:
    return 10


In [ ]:
predictions_df.head(3)

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,Positivo
1,hub93yq,Campeonato de quarto,Neutro
2,hv1xsjp,Não entendi,Neutro


In [ ]:
predictions_df.loc[:, 'prediction'] = predictions_df.loc[:,'prediction'].apply(lambda x: rotulacao(x))
predictions_df.head(3)

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2
1,hub93yq,Campeonato de quarto,1
2,hv1xsjp,Não entendi,1


In [ ]:
merge = pd.merge(sample_comments_df, predictions_df, how='inner', on='id')
#merge = merge[['id',	'Comentário',	'class_result',	'total_agreement', 'prediction']]
#merge.drop('Unnamed: 0.1', inplace=True)
merge.drop('Unnamed: 0', axis=1, inplace=True)
merge.drop('text', axis=1, inplace=True)
merge.rename(columns={"prediction": "gpt_prompt1_fine"}, inplace=True)
merge

,id,Comentário,class_result,total_agreement,gpt_prompt1,gpt_prompt2,abaporu,bertimbau,gpt_prompt1_fine
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1,2,2,2,2
1,hub93yq,Campeonato de quarto,1,1,1,1,1,1,1
2,hv1xsjp,Não entendi,1,1,1,1,1,1,1
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0,0,0,0,0
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...
166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1,1,1,0,1
167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2,2,2,2,2
168,izqc5yj,Lógico que desabou kkkkk,1,0,1,1,0,1,2
169,j0kxoll,como é bom ter democracia,2,1,2,2,2,2,2


In [ ]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
from sklearn.metrics import accuracy_score

#DADOS 90 ft:gpt-4o-2024-08-06:personal::AlJxCMLn

In [ ]:
accuracy_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]))

0.8128654970760234

In [ ]:
f1_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.8148423233666763

In [ ]:
precision_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.831578947368421

In [ ]:
recall_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.8128654970760234

# DADOS 84 ft:gpt-4o-2024-08-06:personal::AlJ2QGgD

In [ ]:
accuracy_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]))

0.8187134502923976

In [ ]:
f1_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.8208878328973369

In [ ]:
precision_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.8381550603630193

In [ ]:
recall_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.8187134502923976

In [ ]:
merge.to_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')

#Dados 78 ft:gpt-4o-2024-08-06:personal::AlIYx1ik

In [ ]:
accuracy_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]))

0.7953216374269005

In [ ]:
f1_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.7977917232538827

In [ ]:
precision_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.8131173882600642

In [ ]:
recall_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.7953216374269005

#DADOS 72 ft:gpt-4o-2024-08-06:personal::AlHSi6ls

In [ ]:
accuracy_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]))

0.7777777777777778

In [ ]:
f1_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.7795230971485563

In [ ]:
precision_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.7936823766733907

In [ ]:
recall_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1_fine"]), average='weighted')

0.7777777777777778

In [ ]:
#merge.to_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')